In [ ]:
struct Interval {
    int lower;
    int upper;
    bool contains(int point) {
        return lower <= point && point < upper;
    }
};

void foo(Interval intv, int point) {
    if (intv.lower <= point && point < intv.upper) {
        // yep
    }
    
    if (intv.contains(point)) {
        // yep
    }
}


# Priority Queue Implementation

## Write a Priority Queue class with `set_priority` functionality

### 9:30 am discussion

Class should be able to:
- `add(item, priority)`
- `set_priority(item, priority)`
- `top`
- `pop`
- `is_empty`


Ideas:
- Use a heap
- Search for an item to change priority
  - Use another structure (BST?) with pointers to items
- Map: key is priority level, value is list of items with that priority
  - Hashmap is fast
    - How do I find things of highest priority
      - Search each possible key, one at a time
  - BST is better
- BST
  - Add is easy - keeps things sorted by priority
    - Can I have duplicate priorities?
      - Store lists of values at a priority 
  - Could you do some AVL stuff after changing priority?
- Set-Priority
  - Remove the old item, add a new version of the item with new priority
    - Can I remove from the middle?
      - Pop only takes top
    - A little more complicated
  - Change the priority of the existing, and percolate
    - Simpler
    - Does this work correctly?
    - Percolate which way?
      - Compare to existing priority
    - Update pointers in lookup table
      - If any item swaps position, update the table with the new positions
  - How do I find the existing value?
    - Search each item in the heap (which is just a linear array)
      - $O(n)$
    - Have a BST map for value=>priority, then search by priority in a BST storing things by priority
      - $O(\log n)$ 
    - Use a hashmap to map value=>position in heap
      - $O(1)$ 
    - 
























Class should be able to:
- `add(item, priority)`
- `set_priority(item, priority)`
- `top`
- `pop`
- `is_empty`

Ideas:
- Probably a heap
- Store tuples of priority + value
- How to implement add?
  - heap: create a tuple and stick it in the back and percolate up
- How to implement set_priority?
  - Need a way to access tuple for a specific item
    - Just root would be easy...
  - Strategy 1: save a new tuple with the value and new priority
    - Delete the old one
      - How?
      - Pop them all off, push onto new heap, filter item of interest
        - Big-O?  slow $O(n \log n)$
      - 
    - Insert new one
  - Strategy 2: Go find the value
    - Compare all items until you find the value
      - Big-O $O(n)$ using BFS
    - Once found, change priority, percolate
    - BST instead of heap
      - Can I store duplicate priorities?
        - Just make the "less than"?
        - Make a node store a list of values
        - Can I search fast by value? or just priority?
  - Strategy 3: maintain a lookup for the values
    - BST for values: $O(\log n)$
    - Hashtable for values: $O(1)$ 
    - Update the table as the heap changes
    
    
    
    
    
    
    
    
    
    
    
    

In [1]:
#include <string>
#include <sstream>
#include <iostream>
#include <map>
#include <vector>
#include <utility>  // swap
using namespace std;

In [94]:
template<class T> class PQ {
    struct Node {
        int priority;
        T value;
        Node(T value, int priority) : priority(priority), value(value) {}
    };
    
    unordered_map<T, int> lookup;
    vector<Node*> array;
    
    int _parent(int const& index) {
        return (index - 1) / 2;
    }
    
    int _left(int const& index) {
        return 2 * index + 1;
    }
    
    void _percolate_up(int const& index) {
        if (index == 0) { return; }
        
        // If the parent is heavier than the item, swap and continue percolating
        int parent = _parent(index);
        if (array.at(parent)->priority > array.at(index)->priority) {
            // Swap values
            swap(array[parent], array[index]);
            
            // Update map
            lookup[array[parent]->value] = parent;
            lookup[array[index]->value] = index;
         
            // Recurse!
            _percolate_up(parent);
        }        
    }

    void _percolate_down(int const& index) {
        
        int left = _left(index);
        int right = left + 1;
        
        // Which child is lighter?
        // I want the child that exists and has the lighter value
        
        if (left >= array.size()) {
            // Left child doesn't exist, so right doesn't either
            return;
        }
        
        // Assume left is lighter, then compare to right, if it exists
        // Use <= to pick right if there is a tie
        int &child = left;
        if (right < array.size() && array[right]->priority <= array[left]->priority) {
            child = right;
        }
        
        // Now compare the parent value to the child value
        if (array[index]->priority > array[child]->priority) {
            // Swap
            swap(array[index], array[child]);
            
            // Update map
            lookup[array[index]->value] = index;
            lookup[array[child]->value] = child;
            
            // Recurse!
            _percolate_down(child);
        }
    }
    
    public:
    
    PQ() : array(), lookup() {}
    
    ~PQ() {
        for (auto nodePtr : array) {
            delete nodePtr;
        }
    }
    
    bool has_item(T const& item) {
        return lookup.find(item) != lookup.end();
    }
    
    bool add(T const& item, int priority) {
        if (has_item(item)) { return false; }
        
        // Add to array
        Node* node = new Node(item, priority);
        
        // Push to back of array
        array.push_back(node);
        int pos = array.size() - 1;
        
        // Add to map
        lookup[item] = pos;

        // Percolate up
        _percolate_up(pos);
        
        return true;
    }
    
    bool set_priority(T item, int priority) {
        // If the item doesn't exist in the queue, return false
        if (!has_item(item)) { return false; }
        
        // Get the node
        int pos = lookup[item];
        
        // If the priority is the same, return false
        if (array[pos]->priority == priority) { return false; }
        
        // Update the priority
        array[pos]->priority = priority;
        
        // Percolate
        _percolate_up(pos);
        _percolate_down(pos);
        
        return true;
    }

    bool is_empty() const {
        return array.empty();
    }
    
    T const& top() const {
        if (is_empty()) { throw invalid_argument("Queue is empty. No item to top!"); }
        return array[0]->value;
    }
    
    void pop() {
        // Delete first
        lookup.erase(array[0]->value);
        
        // Put last item in front
        array[0] = array[array.size()-1];
        lookup[array[0]->value] = 0;
        
        // Remove last
        array.pop_back();
        
        // Percolate down the first
        _percolate_down(0);
    }
    
    string as_array() const {
        stringstream ss;
        for (auto item : array) {
            ss << item->value << " (" << item->priority << "), "; 
        }
        return ss.str();
    }
};

In [95]:
PQ<string> pq;

In [96]:
pq.add("foo", 8);
pq.add("bar", 28);
pq.add("baz", 28);
pq.add("quux", 7);

In [97]:
cout << pq.as_array() << endl;

quux (7), foo (8), baz (28), bar (28), 


In [98]:
pq.set_priority("bar", 6);

In [99]:
cout << pq.as_array() << endl;

bar (6), quux (7), baz (28), foo (8), 


In [100]:
cout << pq.top() << endl;

bar


In [101]:
cout << pq.as_array() << endl << endl;

while (!pq.is_empty()) {
    cout << pq.top() << endl;
    pq.pop();
    cout << pq.as_array() << endl << endl;
}

bar (6), quux (7), baz (28), foo (8), 

bar
quux (7), foo (8), baz (28), 

quux
foo (8), baz (28), 

foo
baz (28), 

baz




# Heap Sort

**We did not discuss this in class, but the content is included for your personal edification.**

- First you heapify
- Then you empty the heap

<div class='big centered'> 🤓 </div>

## Can you use a heap to sort data?

Sure!

Add everything to a heap: $O(n)$  
Pull everything off the heap: $O(n \log n)$

But you can also use heap logic to perform a sort on the items *in place*. 

## Step 1: Heapify

First, impose the **heap ordering property** onto the items in your array.

But reverse the comparison property.

So, if you want to sort least to greatest, make "greater" items percolate **up**.

I.E. if you want to sort ascending values, use a max-heap.

- You partition the array into the "heap" side and the "remainder" side.
- Add each item, one at a time, to the "heap" side of the array
  - After each add, percolate up
  - Once all the items are added, you are done.

```
Add 3.
value: 3 2 4 1 5
index: 1 2 3 4 5
heap:  ^


Add 2.
value: 3 2 4 1 5
index: 1 2 3 4 5
heap:    ^


Add 4.
value: 3 2 4 1 5
index: 1 2 3 4 5
heap:      ^

value: 4 2 3 1 5
index: 1 2 3 4 5
heap:      ^


Add 1.
value: 4 2 3 1 5
index: 1 2 3 4 5
heap:        ^


Add 5.
value: 4 2 3 1 5
index: 1 2 3 4 5
heap:          ^

value: 4 5 3 1 2
index: 1 2 3 4 5
heap:          ^

value: 5 4 3 1 2
index: 1 2 3 4 5
heap:          ^

```

We call this **heapify**.

## Step 2: Empty the Heap

- While the heap is not empty
  - Swap the root with the value in the last position of the array
  - Percolate down

```
value: 5 4 3 1 2
index: 1 2 3 4 5
heap:          ^

Remove 5.
value: 2 4 3 1 5
index: 1 2 3 4 5
heap:        ^

value: 4 2 3 1 5
index: 1 2 3 4 5
heap:        ^


Remove 4.
value: 1 2 3 4 5
index: 1 2 3 4 5
heap:      ^

value: 3 2 1 4 5
index: 1 2 3 4 5
heap:      ^


Remove 3.
value: 1 2 3 4 5
index: 1 2 3 4 5
heap:    ^

value: 2 1 3 4 5
index: 1 2 3 4 5
heap:    ^


Remove 2.
value: 1 2 3 4 5
index: 1 2 3 4 5
heap:  ^

Done!
```



## Heap Sort Intuition

- Use the heap ordering property to put the largest value in the front
- One by one, swap the largest value to the back and reheapify the array